# The best coffee on my way to work in Sydney, Australia

##  Introduction/Business Problem

Eventhough I am working from home now due to the COVID-19 pandemic, I know one day, things will go back to normal (well... semi normal?) and I will get back and work in the Sydney office again! As an addict to caffeine, I am looking forward to my morning coffee made by a professional barista. However, things are changing quickly including cafe and coffee shops. They might have changed ownership, barista, or even blending in a different way. I still remember once I had a bad experience spending like AUD5 for a cup of crap from a coffee shop that used to be my favourite!!! To avoid the same from happening again, I am going to take this opportunity to find out the best coffee shops between my train station and the office. I will run this query again on the day before I am back to the office to make sure it won't ruin my day!

## Overview of how data will be collected and examined

First I am going to find out the coordinates of the Townhall station and my work place. I will take the average of the coordinates as the mid-point, and use Foursquare to search for all the coffee shop/cafe within the radius. Then I will trim it down to the dataset with the name of the streets that I might be passing (I don't mind to go a bit further for a cup of perfect coffee). I believe in that area number of coffee shops will be less than 50 so it satisfy the searh LIMIT in Foursquare. Then, I will run queries to get the ratings of these coffee shops/cafes (venue) and only select the top to further proceed.

I have two particular coffee shops/cafes I enjoy very much with. Next I will make some Premium API Calls to find out tips and users who gave the tip. I will use the ratings done by these users as a criteria to trim down my final decision.

Well with a Personal Tier I have 99,500 Regular API Calls per day and 500 Premium API Calls per day. It should be enough to achieve this little project.

### Step 1, collect data from Foursqure in the area interested

In [3]:
%matplotlib inline

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [35]:
# Visually from google map, Prince Alfred Park is in the middle of the City of Sydney LGA. Let's find out what is the coordinate.
# this is the address of the Prince Alfred Park:
venue_TH = 'Townhall Station'
address_TH = '63 George St, Sydney, Australia'
venue_OF = 'Office'
address_OF = '419 Kent St, Sydney, Australia'
latitude_TH = -33.873251
longitude_TH = 151.206718
latitude_OF = -33.868780
longitude_OF = 151.204501

print(venue_TH)
print('address:',address_TH)
print(latitude_TH, longitude_TH)
print('\n')
print(venue_OF)
print('address:',address_OF)
print(latitude_TH, longitude_OF)
print('\n')
print('This is the middle point:')
latitude_MID = (latitude_TH + latitude_OF)/2
longitude_MID = (longitude_TH + longitude_OF)/2
print(latitude_MID, longitude_MID)


Townhall Station
address: 63 George St, Sydney, Australia
-33.873251 151.206718


Office
address: 419 Kent St, Sydney, Australia
-33.873251 151.204501


This is the middle point:
-33.8710155 151.20560949999998


In [36]:
# Let's calculate the distance bewteen the two points: Townhall and Office

from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373

lat1 = radians(latitude_TH)
lon1 = radians(longitude_TH)
lat2 = radians(latitude_OF)
lon2 = radians(longitude_OF)

dlon = lon2 - lon1
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c *1000

print("Result:", distance,'m')

Result: 537.8080068825911 m


In [54]:
# let's get ready to retrieve data via API from Foursquare
CLIENT_ID = 'BESBPEDHNTKUY32TFLTWUP3HLLAG45FNSH4DB1PQI2KGPJP2' #  Foursquare ID
CLIENT_SECRET = 'MMB0U1FGJMTCDWEHNOI1YSWMKCGT1E0RBRH3MBBFKNEXLO2M' #  Foursquare Secret
VERSION = '20180604'
LIMIT = 50 #50 is the maximum limit anyway

# search radius is half of the distance between Townhall station and the office
search_query = 'coffee'
radius_search = distance / 2

In [38]:
# Let's define the search query.
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    latitude_MID,
    longitude_MID,
    VERSION,
    search_query,
    radius_search,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=BESBPEDHNTKUY32TFLTWUP3HLLAG45FNSH4DB1PQI2KGPJP2&client_secret=MMB0U1FGJMTCDWEHNOI1YSWMKCGT1E0RBRH3MBBFKNEXLO2M&ll=-33.8710155,151.20560949999998&v=20180604&query=coffee&radius=268.9040034412956&limit=500'

In [39]:
results_all_coffee = requests.get(url).json()

In [98]:
# Get relevant part of JSON and transform it into a *pandas* dataframe
# assign relevant part of JSON to venues
venues_all_coffee = results_all_coffee['response']['venues']

# tranform venues into a dataframe
df_raw = json_normalize(venues_all_coffee)

In [173]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_raw.columns if col.startswith('location.')] + ['id']
df_filtered = df_raw.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]
df_filtered.head(50)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Foodoo Coffee Lounge,Coffee Shop,MYER 5th Floor (Pitt Street ),AU,Sydney,Australia,George Street,195,[MYER 5th Floor (Pitt Street ) (George Street)...,"[{'label': 'display', 'lat': -33.8703641546462...",-33.870364,151.207573,NaN,2000,NSW,51d7c52a498edf6da91d6007
1,Gumption by Coffee Alchemy,Coffee Shop,"Shop 11, 412-414 George St.",AU,Sydney,Australia,in Strand Arcade,260,"[Shop 11, 412-414 George St. (in Strand Arcade...","[{'label': 'display', 'lat': -33.8694396595288...",-33.869440,151.207700,NaN,2000,NSW,529bb94d498e6a0cd597b7ca
2,Old Vienna Coffee House,Coffee Shop,455 George St.,AU,Sydney,Australia,NaN,106,"[455 George St., Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8713127062593...",-33.871313,151.206705,NaN,2000,NSW,4b18474af964a5201cd023e3
3,Kent St Specialty Coffee,Café,414 Kent St.,AU,Sydney,Australia,Market St.,114,"[414 Kent St. (Market St.), Sydney NSW 2000, A...","[{'label': 'display', 'lat': -33.8718974149045...",-33.871897,151.204972,NaN,2000,NSW,4d8aae4d26a36ea8f39cfaad
4,Campos Coffee,Café,NaN,AU,NaN,Australia,NaN,266,[Australia],"[{'label': 'display', 'lat': -33.8731851049854...",-33.873185,151.206832,NaN,NaN,NaN,51440250e4b0ab7c89300305
5,The Australian Coffee School,Trade School,"Level 5, 250 Pitt St",AU,Sydney,Australia,Park St,331,"[Level 5, 250 Pitt St (Park St), Sydney NSW 20...","[{'label': 'display', 'lat': -33.8730225267590...",-33.873023,151.208255,NaN,2000,NSW,5004ab83e4b06c76d1e31d37
6,Veneziano Caffee Speciality Coffee,Café,NaN,AU,Sydney,Australia,Kent Street,94,"[Kent Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8716605522493...",-33.871661,151.204942,NaN,2000,NSW,4f690c36e4b07a700b357b30
7,Mission Coffee,Café,NaN,AU,NaN,Australia,NaN,104,[Australia],"[{'label': 'display', 'lat': -33.8702140747043...",-33.870214,151.206195,NaN,NaN,NaN,55cbd122498eebb48c50f8e6
8,Gloria Jean's Coffees,Coffee Shop,The Galleries Victoria\rCnr Pitt & Park Streets,AU,Sydney,Australia,cnr Pitt St. & Park St.,180,[The Galleries Victoria\rCnr Pitt & Park Stree...,"[{'label': 'display', 'lat': -33.870428, 'lng'...",-33.870428,151.207425,NaN,2000,NSW,4b5aa658f964a5206bcf28e3
9,Market St Coffee,Coffee Shop,NaN,AU,Sydney,Australia,NaN,138,"[Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.870742, 'lng'...",-33.870742,151.207068,NaN,2000,NSW,59090e7ea6fe4d71fc743edc


### Step 2, trim down number of coffee shops/cafes to those I may visit

In [174]:
# let's drop the Coffee schools
df_filtered.drop(df_filtered.index[[5,11,21]], inplace = True)
df_filtered = df_filtered.reset_index(drop=True)

In [175]:
df_filtered.head(30)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Foodoo Coffee Lounge,Coffee Shop,MYER 5th Floor (Pitt Street ),AU,Sydney,Australia,George Street,195,[MYER 5th Floor (Pitt Street ) (George Street)...,"[{'label': 'display', 'lat': -33.8703641546462...",-33.870364,151.207573,NaN,2000,NSW,51d7c52a498edf6da91d6007
1,Gumption by Coffee Alchemy,Coffee Shop,"Shop 11, 412-414 George St.",AU,Sydney,Australia,in Strand Arcade,260,"[Shop 11, 412-414 George St. (in Strand Arcade...","[{'label': 'display', 'lat': -33.8694396595288...",-33.869440,151.207700,NaN,2000,NSW,529bb94d498e6a0cd597b7ca
2,Old Vienna Coffee House,Coffee Shop,455 George St.,AU,Sydney,Australia,NaN,106,"[455 George St., Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8713127062593...",-33.871313,151.206705,NaN,2000,NSW,4b18474af964a5201cd023e3
3,Kent St Specialty Coffee,Café,414 Kent St.,AU,Sydney,Australia,Market St.,114,"[414 Kent St. (Market St.), Sydney NSW 2000, A...","[{'label': 'display', 'lat': -33.8718974149045...",-33.871897,151.204972,NaN,2000,NSW,4d8aae4d26a36ea8f39cfaad
4,Campos Coffee,Café,NaN,AU,NaN,Australia,NaN,266,[Australia],"[{'label': 'display', 'lat': -33.8731851049854...",-33.873185,151.206832,NaN,NaN,NaN,51440250e4b0ab7c89300305
5,Veneziano Caffee Speciality Coffee,Café,NaN,AU,Sydney,Australia,Kent Street,94,"[Kent Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8716605522493...",-33.871661,151.204942,NaN,2000,NSW,4f690c36e4b07a700b357b30
6,Mission Coffee,Café,NaN,AU,NaN,Australia,NaN,104,[Australia],"[{'label': 'display', 'lat': -33.8702140747043...",-33.870214,151.206195,NaN,NaN,NaN,55cbd122498eebb48c50f8e6
7,Gloria Jean's Coffees,Coffee Shop,The Galleries Victoria\rCnr Pitt & Park Streets,AU,Sydney,Australia,cnr Pitt St. & Park St.,180,[The Galleries Victoria\rCnr Pitt & Park Stree...,"[{'label': 'display', 'lat': -33.870428, 'lng'...",-33.870428,151.207425,NaN,2000,NSW,4b5aa658f964a5206bcf28e3
8,Market St Coffee,Coffee Shop,NaN,AU,Sydney,Australia,NaN,138,"[Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.870742, 'lng'...",-33.870742,151.207068,NaN,2000,NSW,59090e7ea6fe4d71fc743edc
9,Speed Bar coffee Shop,Coffee Shop,29 Park Street,AU,Sydney,Australia,NaN,275,"[29 Park Street, Sydney NSW 2010, Australia]","[{'label': 'display', 'lat': -33.8731453913815...",-33.873145,151.207125,NaN,2010,NSW,4d70758095acb1f746ab06b9


In [176]:
# let's have a look on what we've got so far on a map.

coffee_map = folium.Map(location=[latitude_MID, longitude_MID], zoom_start=16) # generate map centred around the mid point

# add red circle markers to represent the Train Station and the Office
folium.features.CircleMarker(
    [latitude_TH, longitude_TH],
    radius=10,
    color='red',
    popup='Townhall Station',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_map)

folium.features.CircleMarker(
    [latitude_OF, longitude_OF],
    radius=10,
    color='red',
    popup='Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_map)

# add big circle represent the the search area in Foursquare

folium.Marker([latitude_MID, longitude_MID]).add_to(coffee_map)
folium.Circle([latitude_MID, longitude_MID],
                    radius=radius_search
                   ).add_to(coffee_map)

# add the coffee shop and Cafe as blue circle markers
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(coffee_map)

# display map
coffee_map


In [177]:
# I would like to avoid going to the eastern side of George Street (the nine dots with the largest longtitude)
# also I don't want to go Sussex street and anywhere further West (the two dots with the least longtitude)
# and also not anywhere South of Townhall (the 3 red dot, latitude < latitude_MID)

In [178]:
# sort data frame as per latitude
df_filtered = df_filtered.sort_values(by = 'lat')
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Tom n Toms Coffee,Coffee Shop,97-99 Bathurst St.,AU,Sydney,Australia,NaN,358,"[97-99 Bathurst St., Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8742143902610...",-33.874214,151.206011,NaN,2000,NSW,4b0b1d80f964a520252d23e3
1,Adore Coffee,Coffee Shop,NaN,AU,NaN,Australia,NaN,328,[Australia],"[{'label': 'display', 'lat': -33.8739581932831...",-33.873958,151.205284,NaN,NaN,NaN,51ce302a498e48b06732c561
2,Breakfast Coffee,None,"Shop G13, Wynyard Station",AU,NaN,Australia,NaN,318,"[Shop G13, Wynyard Station, Australia]","[{'label': 'display', 'lat': -33.8737, 'lng': ...",-33.873700,151.206810,NaN,NaN,NaN,4e4b4064483bb8f059908442
3,Campos Coffee,Café,NaN,AU,NaN,Australia,NaN,266,[Australia],"[{'label': 'display', 'lat': -33.8731851049854...",-33.873185,151.206832,NaN,NaN,NaN,51440250e4b0ab7c89300305
4,Speed Bar coffee Shop,Coffee Shop,29 Park Street,AU,Sydney,Australia,NaN,275,"[29 Park Street, Sydney NSW 2010, Australia]","[{'label': 'display', 'lat': -33.8731453913815...",-33.873145,151.207125,NaN,2010,NSW,4d70758095acb1f746ab06b9


In [179]:
df_filtered.drop(df_filtered.index[[0,1,2]], inplace = True)


In [180]:
# sort data frame as per latitude
df_filtered = df_filtered.sort_values(by = 'lng')
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.head(30)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Pyrmont Bridge Coffee Stand,Coffee Shop,114 Cockle Bay Wharf,AU,Sydney,Australia,NaN,322,"[114 Cockle Bay Wharf, Sydney NSW 2000, Austra...","[{'label': 'display', 'lat': -33.8709612414515...",-33.870961,151.202119,NaN,2000,NSW,4dd19cd7c65b9d071f89f79f
1,Harrogate Teas and Coffees,Tea Room,3 Union St.,AU,Pyrmont,Australia,NaN,313,"[3 Union St., Pyrmont NSW 2009, Australia]","[{'label': 'display', 'lat': -33.8686858381798...",-33.868686,151.203696,NaN,2009,NSW,4b19c425f964a520c4e323e3
2,Vittoria Coffee,Café,NaN,AU,NaN,Australia,NaN,259,[Australia],"[{'label': 'display', 'lat': -33.8689596473147...",-33.868960,151.204282,NaN,NaN,NaN,509b23cee4b04e2b3d9bf68f
3,Adore Estate Coffee,Coffee Shop,Clarence,AU,Sydney,Australia,NaN,248,"[Clarence, Sydney NSW, Australia]","[{'label': 'display', 'lat': -33.8689828088546...",-33.868983,151.204498,NaN,NaN,NSW,4ee7cddb9a5288af4817de01
4,Skittle Lane Coffee,Coffee Shop,40 King St,AU,Sydney,Australia,NaN,266,"[40 King St, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8687580721696...",-33.868758,151.204657,Wynyard,2000,NSW,56258f2d498e93403498716c
5,Veneziano Caffee Speciality Coffee,Café,NaN,AU,Sydney,Australia,Kent Street,94,"[Kent Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8716605522493...",-33.871661,151.204942,NaN,2000,NSW,4f690c36e4b07a700b357b30
6,Kent St Specialty Coffee,Café,414 Kent St.,AU,Sydney,Australia,Market St.,114,"[414 Kent St. (Market St.), Sydney NSW 2000, A...","[{'label': 'display', 'lat': -33.8718974149045...",-33.871897,151.204972,NaN,2000,NSW,4d8aae4d26a36ea8f39cfaad
7,Danes Gourmet Coffee,Coffee Shop,171 Clarence Street,AU,Sydney,Australia,NaN,281,"[171 Clarence Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.86854, 'lng':...",-33.868540,151.205010,NaN,2000,NSW,5bcfc5281c675b002cfdfc15
8,KB Coffee Roasters,Coffee Shop,NaN,AU,NaN,Australia,NaN,227,[Australia],"[{'label': 'display', 'lat': -33.868986, 'lng'...",-33.868986,151.205304,NaN,NaN,NaN,4f83669ce4b0cfa53e2e9e5a
9,Pure Grind Café,Café,225 Clarence Street,AU,Sydney,Australia,NaN,54,"[225 Clarence Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8705260317789...",-33.870526,151.205550,Sydney City Center,2000,NSW,4b6f3173f964a5205be32ce3


In [181]:
df_filtered.drop(df_filtered.index[[0,1,18,19,20,21,22,23,24,25,26]], inplace = True)
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.head(30)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Vittoria Coffee,Café,NaN,AU,NaN,Australia,NaN,259,[Australia],"[{'label': 'display', 'lat': -33.8689596473147...",-33.868960,151.204282,NaN,NaN,NaN,509b23cee4b04e2b3d9bf68f
1,Adore Estate Coffee,Coffee Shop,Clarence,AU,Sydney,Australia,NaN,248,"[Clarence, Sydney NSW, Australia]","[{'label': 'display', 'lat': -33.8689828088546...",-33.868983,151.204498,NaN,NaN,NSW,4ee7cddb9a5288af4817de01
2,Skittle Lane Coffee,Coffee Shop,40 King St,AU,Sydney,Australia,NaN,266,"[40 King St, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8687580721696...",-33.868758,151.204657,Wynyard,2000,NSW,56258f2d498e93403498716c
3,Veneziano Caffee Speciality Coffee,Café,NaN,AU,Sydney,Australia,Kent Street,94,"[Kent Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8716605522493...",-33.871661,151.204942,NaN,2000,NSW,4f690c36e4b07a700b357b30
4,Kent St Specialty Coffee,Café,414 Kent St.,AU,Sydney,Australia,Market St.,114,"[414 Kent St. (Market St.), Sydney NSW 2000, A...","[{'label': 'display', 'lat': -33.8718974149045...",-33.871897,151.204972,NaN,2000,NSW,4d8aae4d26a36ea8f39cfaad
5,Danes Gourmet Coffee,Coffee Shop,171 Clarence Street,AU,Sydney,Australia,NaN,281,"[171 Clarence Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.86854, 'lng':...",-33.868540,151.205010,NaN,2000,NSW,5bcfc5281c675b002cfdfc15
6,KB Coffee Roasters,Coffee Shop,NaN,AU,NaN,Australia,NaN,227,[Australia],"[{'label': 'display', 'lat': -33.868986, 'lng'...",-33.868986,151.205304,NaN,NaN,NaN,4f83669ce4b0cfa53e2e9e5a
7,Pure Grind Café,Café,225 Clarence Street,AU,Sydney,Australia,NaN,54,"[225 Clarence Street, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8705260317789...",-33.870526,151.205550,Sydney City Center,2000,NSW,4b6f3173f964a5205be32ce3
8,Klink Handmade Espresso,Coffee Shop,281 Clarence St,AU,Sydney,Australia,nr Druitt St,146,"[281 Clarence St (nr Druitt St), Sydney NSW 20...","[{'label': 'display', 'lat': -33.872331, 'lng'...",-33.872331,151.205593,NaN,2000,NSW,4b8b0961f964a520858f32e3
9,Vella Nero,Café,259 Clarence St.,AU,Sydney,Australia,btwn Market St. & Druitt St.,84,[259 Clarence St. (btwn Market St. & Druitt St...,"[{'label': 'display', 'lat': -33.8717699813282...",-33.871770,151.205665,NaN,2000,NSW,4b09f05df964a520242023e3


In [182]:
# let's have a look on what we've got so far on a map.

coffee_map = folium.Map(location=[latitude_MID, longitude_MID], zoom_start=16) # generate map centred around the mid point

# add red circle markers to represent the Train Station and the Office
folium.features.CircleMarker(
    [latitude_TH, longitude_TH],
    radius=10,
    color='red',
    popup='Townhall Station',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_map)

folium.features.CircleMarker(
    [latitude_OF, longitude_OF],
    radius=10,
    color='red',
    popup='Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_map)

# add big circle represent the the search area in Foursquare

folium.Marker([latitude_MID, longitude_MID]).add_to(coffee_map)
folium.Circle([latitude_MID, longitude_MID],
                    radius=radius_search
                   ).add_to(coffee_map)

# add the coffee shop and Cafe as blue circle markers
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(coffee_map)

# display map
coffee_map


In [183]:
# now I am happy with the coffe shops I have. Let's rename the dataframe.
df_coffee = df_filtered
df_coffee.shape

(16, 16)

### Step 3, get the ratings of these 16 coffee shops